In [36]:
from datasets import load_dataset
from transformers import LayoutLMv2ImageProcessor, LayoutXLMTokenizerFast
from transformers.models.lilt.modeling_lilt import LiltForPretraining
import torch
from data_collator import DataCollatorForLiltPretraining
from tqdm import tqdm
import numpy as np

In [2]:
model = LiltForPretraining.from_pretrained("SCUT-DLVCLab/lilt-infoxlm-base")
# model = model.cuda()

Some weights of LiltForPretraining were not initialized from the model checkpoint at SCUT-DLVCLab/lilt-infoxlm-base and are newly initialized: ['lm_head.mvlm_head.dense.weight', 'lm_head.kpl_head.decoder.weight', 'lm_head.cai_head.dense.bias', 'lm_head.kpl_head.layer_norm.bias', 'lm_head.cai_head.layer_norm.weight', 'lm_head.kpl_head.bias', 'lm_head.kpl_head.dense.weight', 'lm_head.mvlm_head.layer_norm.weight', 'lm_head.kpl_head.layer_norm.weight', 'lm_head.cai_head.layer_norm.bias', 'lm_head.cai_head.decoder.weight', 'lm_head.mvlm_head.bias', 'lm_head.mvlm_head.layer_norm.bias', 'lm_head.cai_head.dense.weight', 'lm_head.mvlm_head.dense.bias', 'lm_head.cai_head.bias', 'lm_head.cai_head.decoder.bias', 'lm_head.kpl_head.dense.bias', 'lm_head.kpl_head.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
rvl_cdip = load_dataset('rvl_cdip',split='train',keep_in_memory=True)
rvl_cdip

Found cached dataset rvl_cdip (/home/hj36wegi/scratch/data/huggingface/datasets/rvl_cdip/default/1.0.0/ea410993ed3f5b9744d8616ffbaad5f70a75a21a4233626dd07b3de31d381e53)


Dataset({
    features: ['image', 'label'],
    num_rows: 320000
})

In [43]:
LayoutLMv2ImageProcessor?

Init signature:
LayoutLMv2ImageProcessor(
    do_resize: bool = True,
    size: Dict[str, int] = None,
    resample: PIL.Image.Resampling = <Resampling.BILINEAR: 2>,
    apply_ocr: bool = True,
    ocr_lang: Optional[str] = None,
    tesseract_config: Optional[str] = '',
    **kwargs,
) -> None
Docstring:     
Constructs a LayoutLMv2 image processor.

Args:
    do_resize (`bool`, *optional*, defaults to `True`):
        Whether to resize the image's (height, width) dimensions to `(size["height"], size["width"])`. Can be
        overridden by `do_resize` in `preprocess`.
    size (`Dict[str, int]` *optional*, defaults to `{"height": 224, "width": 224}`):
        Size of the image after resizing. Can be overridden by `size` in `preprocess`.
    resample (`PILImageResampling`, *optional*, defaults to `PILImageResampling.BILINEAR`):
        Resampling filter to use if resizing the image. Can be overridden by the `resample` parameter in the
        `preprocess` method.
    apply_ocr (`bool`

In [4]:
img_processor = LayoutLMv2ImageProcessor(do_resize=False,
                                         do_rescale=False,
                                         do_normalize=False,
                                         ocr_lang='eng',
                                         )
tokenizer = LayoutXLMTokenizerFast.from_pretrained("SCUT-DLVCLab/lilt-infoxlm-base")

In [5]:
def process_example(example):
    
    images = [img.convert('RGB') for img in example['image']]
    
    features = img_processor(images)
    encoding = tokenizer(
        text=features["words"],
        boxes=features["boxes"],
        return_special_tokens_mask=True,
        return_tensors='pt',
        max_length=512,
        padding='max_length',
        truncation=True
    )   
    return encoding

In [20]:
def process_bounding_boxes(batch):

    get_box_number = lambda x,y: int(y*10 + x) # Get the box number from the x and y coordinates of 10x10 grid

    bs,seq,_ = batch['bbox'].shape
    all_boxes = batch['bbox'].view(-1,4)
    bbox_loc = []

    for box in all_boxes:
        tl_x,tl_y,br_x,br_y = box.numpy()
        ce_x = (tl_x + br_x)/2 # Find the center of the box
        ce_y = (tl_y + br_y)/2

        tl_x,tl_y,br_x,br_y,ce_x,ce_y = int(tl_x/100),int(tl_y/100),int(br_x/100),int(br_y/100),int(ce_x/100),int(ce_y/100)
        tl = get_box_number(tl_x,tl_y)
        br = get_box_number(br_x,br_y)
        ce = get_box_number(ce_x,ce_y)

        bbox_loc.append((tl,br,ce))

    batch['bbox_loc'] = torch.LongTensor(bbox_loc).view(bs,seq,-1) # (bs,seq,4)
    return batch

In [21]:
def processor(example):
    batch = process_example(example)
    batch = process_bounding_boxes(batch)
    return batch

In [22]:
rvl_cdip.set_transform(processor)

In [34]:
rvl_cdip[1953]['bbox_loc'][rvl_cdip[1953]['bbox_loc']>99]

tensor([102, 102, 102, 102, 102, 110, 110, 110])

In [39]:
x = np.nonzero(rvl_cdip[1953]['bbox_loc'].numpy()>99)

In [42]:
rvl_cdip[1953]['bbox'][x[0]]

tensor([[  65,  988,  241, 1000],
        [  65,  988,  241, 1000],
        [  65,  988,  241, 1000],
        [  65,  988,  241, 1000],
        [  65,  988,  241, 1000],
        [1000, 1000, 1000, 1000],
        [1000, 1000, 1000, 1000],
        [1000, 1000, 1000, 1000]])

In [ ]:
bboxes = []
for idx,ex in tqdm(enumerate(rvl_cdip),total=len(rvl_cdip)):
    bboxes.append(ex['bbox_loc'])

In [7]:
collator = DataCollatorForLiltPretraining(tokenizer=tokenizer,seq_len=512)

In [10]:
rvl_cdip2 = rvl_cdip.select(range(100)).map(lambda e: e,batched=True,batch_size=10)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [19]:
collator(rvl_cdip[:10])

AttributeError: 'tokenizers.Encoding' object has no attribute 'keys'

In [8]:
rvl_cdip = rvl_cdip.map(collator,batched=True,batch_size=10)

Map:   0%|          | 0/320000 [00:00<?, ? examples/s]

AttributeError: 'tokenizers.Encoding' object has no attribute 'keys'

In [5]:
lang_adapter_config = HarrysInvConfig(text_output_adapter=True, layout_output_adapter=False)
layout_adapter_config = HarrysConfig(text_output_adapter=False, layout_output_adapter=True)
task_adapter_config = HarrysConfig(text_output_adapter=True, layout_output_adapter=False)

model = LiltForPretraining.from_pretrained("SCUT-DLVCLab/lilt-infoxlm-base")#.from_pretrained("nielsr/lilt-xlm-roberta-base")
model.add_adapter("layout", layout_adapter_config)
model.add_adapter("language", lang_adapter_config)
model.add_adapter("task", task_adapter_config)
model.train_adapter(["layout","task", "language"])
model.freeze_model(True)
model = model.cuda()

Some weights of LiltForPretraining were not initialized from the model checkpoint at SCUT-DLVCLab/lilt-infoxlm-base and are newly initialized: ['lm_head.cai_head.dense.weight', 'lm_head.kpl_head.dense.bias', 'lm_head.kpl_head.decoder.bias', 'lm_head.cai_head.layer_norm.weight', 'lm_head.mvlm_head.dense.weight', 'lm_head.cai_head.bias', 'lm_head.mvlm_head.layer_norm.weight', 'lm_head.mvlm_head.bias', 'lm_head.kpl_head.bias', 'lm_head.mvlm_head.layer_norm.bias', 'lm_head.cai_head.layer_norm.bias', 'lm_head.kpl_head.layer_norm.weight', 'lm_head.kpl_head.decoder.weight', 'lm_head.cai_head.decoder.weight', 'lm_head.kpl_head.layer_norm.bias', 'lm_head.kpl_head.dense.weight', 'lm_head.cai_head.dense.bias', 'lm_head.mvlm_head.dense.bias', 'lm_head.cai_head.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
for batch_idx,batch in tqdm(enumerate(trainer),total=len(trainer)):
    if batch_idx in [80000]:
        print(f"skipping..{batch_idx}")
        continue
    batch = {k:v.cuda() for k,v in batch.items()}
    out = model(**batch)

  0%|▋                                                                                                                                                              | 349/80000 [02:21<8:59:47,  2.46it/s]


RuntimeError: philox_cuda_state for an unexpected CUDA generator used during capture. In regions captured by CUDA graphs, you may only use the default CUDA RNG generator on the device that's current when capture begins. If you need a non-default (user-supplied) generator, or a generator on another device, please file an issue.

In [ ]:
batch['bbox_loc'].max(), batch['input_labels'].max()

In [ ]:
model.config.vocab_size

In [ ]:
len(trainer)

In [ ]:
model.cuda?